In [ ]:
{-# LANGUAGE DeriveFunctor, DeriveFoldable, DeriveTraversable, DeriveGeneric, FlexibleInstances, LambdaCase #-}
-- | Untyped lambda calculus using bound library
import Control.Monad
import Control.Monad.Trans.State
import Data.List
import Text.PrettyPrint
import GHC.Generics (Generic)

import Bound
import Prelude.Extras


class Pretty a where
  pretty :: a -> Doc
instance Pretty Char   where pretty = char
instance Pretty Int    where pretty = int
instance Pretty [Char] where pretty = text

In [ ]:
data Exp a
  = V a
  | Exp a :@ Exp a
  | Lam (Scope () Exp a)
--  | Bot
  deriving (Show,Eq,Functor,Foldable,Traversable,Generic)
instance Show1 Exp
instance Eq1   Exp

instance Applicative Exp where
  pure = V
  (<*>) = ap

instance Monad Exp where
  return = pure
  V a      >>= f = f a
  (a :@ b) >>= f = (a >>= f) :@ (b >>= f)
  Lam lam  >>= f = Lam (lam >>>= f)  
--  Bot      >>= _ = Bot
  

lam :: Eq a => a -> Exp a -> Exp a
lam v b = Lam (abstract1 v b)

infixr 0 !
(!) :: Eq a => a -> Exp a -> Exp a
(!) = lam

lams :: Eq a => [a] -> Exp a -> Exp a
lams = flip $ foldr lam

Now we need to define pretty printer. In some sense it's easy but if we don't want to see excessive number of parens.

In [ ]:
-- Full brackets pretty-printyer
instance Pretty a => Pretty (Exp a) where
  pretty = flip evalState varnames . go False . fmap Left
    where
      go par = \case
        V (Left  e) -> return $ pretty e
        V (Right e) -> return $ pretty e
--        Bot         -> return $ text "BOT"
        f :@ x      -> do
          sf <- go True f
          sx <- go True x
          return $ withParens par $ hcat [sf, text " " , sx]
        e@Lam{}-> do 
          (binds,str) <- goLam e
          return $ withParens par $ hcat [text "\\", hcat (intersperse space binds),text " -> ",str]
      --
      withParens True  = parens
      withParens False = id
      --
      goLam = \case
        Lam lam -> do
          v:vs <- get
          put vs
          (binds,str) <- goLam $ instantiate1 (V (Right v)) lam
          return (pretty v:binds, str)
        e -> do {s <- go False e; return ([],s)}
      --  
      varnames = ['a' .. 'z']

In [ ]:
-- Reduce expression to weak head normal form
whnf :: Exp a -> Exp a
whnf (f :@ a) = case whnf f of
  Lam b -> whnf (instantiate1 a b)
  f'    -> f' :@ a
whnf e = e

-- | Compute the normal form of an expression
nf :: Exp a -> Exp a
nf (Lam b)  = Lam $ toScope $ nf $ fromScope b
nf (f :@ a) = case whnf f of
  Lam b -> nf (instantiate1 a b)
  f'    -> nf f' :@ nf a
nf e = e

-- | Compute normal form of an expression using applicative reduction
appNF :: Exp a -> Exp a
appNF (Lam b) = undefined
appNF (f :@ a) = case whnf f of

In [ ]:
fun1 = lam 'x' (lam 'y' $ V 'x' :@ V 'y')
fun1
pretty fun1
pretty (lams ['x','y'] (V 'x' :@ V 'y'))

In [ ]:
zeroF = lams [    "z","s"] $ V "z"
succF = lams ["n","z","s"] $ V "s" :@ (V "n" :@ V "z" :@ V "s")

ch1 = succF :@ zeroF
ch2 = succF :@ ch1
ch3 = succF :@ ch2

In [ ]:
pretty zeroF
pretty succF

pretty ch1
pretty (nf ch1)
pretty ch2
pretty (nf ch2)
pretty ch3
pretty (nf ch3)


In [ ]:
botTest = lams ['a','b'] (V 'b')
pretty botTest
pretty $ nf $ botTest :@ (error "A")

In [ ]:
omega = lam 'x' (V 'x' :@ V 'x') :@ lam 'x' (V 'x' :@ V 'x')
pretty omega
pretty $ nf omega